In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import tensorflow as tf
from tf_keras_vis.utils import num_of_gpus

_, gpus = num_of_gpus()
print('{} GPUs'.format(gpus))

import tensorflow as tf
print('tensorflow version:{}'.format(tf.__version__))
import tensorflow as tf
print(tf.test.is_gpu_available())

In [ ]:
from keras.preprocessing import image
import numpy as np
import cv2,os, keras
from keras.models import load_model

SavePath = 'xxx'
ModelName = 'xxx'
base_dir = 'xxx'
train_dir = os.path.join(base_dir, 'train/')
validation_dir = os.path.join(base_dir, 'validation/')
IMAGES_FORMAT = ['.jpg']

model = load_model(SavePath+ModelName+'.hdf5')
model.summary()

In [ ]:
import matplotlib.pyplot as plt
from keras.preprocessing import image

def loss(output):
    return (output[0,0])

def model_modifier(m):
    m.layers[-1].activation = tf.keras.activations.linear
    return m

from tensorflow.keras import backend as K
from tf_keras_vis.saliency import Saliency
from tf_keras_vis.utils import normalize

def saliency(img_path,filename,sorf):
    print(img_path)
    img = image.load_img(img_path, target_size=(299, 299))
    images = np.asarray([np.array(img)])
    x = images/255
    y = image.img_to_array(img)
    y = np.expand_dims(y, axis=0)
    y /= 255
    pred=model.predict(y)
    print(pred)
    saliency = Saliency(model,model_modifier=model_modifier,clone=False)
    saliency_map = saliency(loss, x,smooth_samples=20, smooth_noise=0.20)
    saliency_map = normalize(saliency_map)
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.imshow(saliency_map[0], cmap='jet')
    saliency_map = np.uint8(255 * saliency_map[0])
    saliency_map = cv2.applyColorMap(saliency_map, cv2.COLORMAP_JET)
    path = SavePath+'saliency/'+sorf
    cv2.imwrite(path+'/'+filename[:-4]+' p='+str(pred[0,1])+'Saliency_map.jpg', saliency_map)
    superimposed_img = saliency_map * 0.4 + img
    cv2.imwrite(path+'/'+filename[:-4]+' p='+str(pred[0,1])+'Superimposed.jpg', superimposed_img)

vali_fail_dir = validation_dir+'BA/'
fileDir=os.listdir(vali_fail_dir)

for m in range(len(fileDir)):
    saliency(vali_fail_dir+fileDir[m],fileDir[m],'BA')
    
vali_succ_dir = validation_dir+'OS/'
fileDir=os.listdir(vali_succ_dir)

for m in range(len(fileDir)):
    saliency(vali_succ_dir+fileDir[m],fileDir[m],'OS')